In [ ]:
import random
import os
import time
import tweepy
import datetime
import json
import pandas as pd
import csv
import matplotlib as mpl
from geopy.geocoders import Nominatim
import folium
import pickle
import re

In [ ]:
ACCESS_TOKEN = "1214290028701970433-kEuAt1sO18gYwxohZh5l9a2NnJQi8I"
ACCESS_SECRET = "sjWdkpCvzSzWD1JTTNIPDCRgxZQRhHgFs033lAQT5p27c"
CONSUMER_KEY = "8lsE3g44643tDBEj4MSAbQ6rh"
CONSUMER_SECRET = "xTFXRE1ZUloYdtAFrkJqIfe4xUmZ1Q18mEKfOiE2VvEjPGSyAs"

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
auth.secure = True
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

In [ ]:
startDate = datetime.datetime(2020, 2, 1, 0, 0, 0)
endDate =   datetime.datetime(2020, 3, 6, 23, 59, 59)

In [ ]:
handles_list = ["@DelhiPolice", "@DCPNewDelhi", "@DCPSEastDelhi", "@DCPWestDelhi", "@DCPEastDelhi", "@DCPSouthDelhi", "@DCPNEastDelhi", "@dcp_southwest", "@DcpNorthDelhi", "@DCPNWestDelhi", "@CPDelhi", "@DCP_DelhiMetro"]

In [ ]:
mega_list = []

In [ ]:
# THIS FUNCTION IS NOT BEING USED
def mentions(username):
  searchQuery = username
  retweet_filter='-filter:retweets'
  q=searchQuery+retweet_filter
  tweetsPerQry = 100
  fName = username + '_tweets.csv'
  max_id = 1235893284276301825
  sinceId = 1223467264562753538
  maxTweets = 50000 
  tweetCount = 0
  megalist = []
  tweets = []
  o_file = open(fName, 'w')
  f = csv.writer(o_file, delimiter =',')
  f.writerow(["Date", "ID", "Text", "Likes", "Retweets"])
  print("Downloading max {0} tweets".format(maxTweets))
  while tweetCount < maxTweets:
      try:
          if (max_id <= 0):
              if (not sinceId):
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
              else:
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                          since_id=sinceId)
          else:
              if (not sinceId):
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                          max_id=str(max_id - 1))
              else:
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                          max_id=str(max_id - 1),
                                          since_id=sinceId)
          if not new_tweets:
              print("No more tweets found")
              break
    
          for tweet in new_tweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
              tweets.append(tweet)

          for tweet in new_tweets:
              f.writerow([tweet.created_at,tweet.id, tweet.text, tweet.favorite_count, tweet.retweet_count])
          tweetCount += len(new_tweets)
          print("Downloaded {0} tweets".format(tweetCount))
          max_id = new_tweets[-1].id
      except tweepy.TweepError as e:
          # Just exit if any error
          print("some error : " + str(e))
          break

  print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

In [ ]:
def get_tweets(username):
  startDate = datetime.datetime(2020, 2, 1, 0, 0, 0)
  endDate =   datetime.datetime(2020, 3, 6, 23, 59, 59)
  tweets = []
  tmpTweets = api.user_timeline(username)
  for tweet in tmpTweets:
      if tweet.created_at < endDate and tweet.created_at > startDate:
          tweets.append(tweet)

  while (tmpTweets[-1].created_at > startDate):
      tmpTweets = api.user_timeline(username, max_id = tmpTweets[-1].id)
      for tweet in tmpTweets:
          if tweet.created_at < endDate and tweet.created_at > startDate:
              tweets.append(tweet)
  
  return tweets

  # tweet_file = open(username + "_tweets_file.bin","wb")
  # pickle.dump(tweets,tweet_file)
  # tweet_file.close()

In [ ]:
# Get tweets from twitter
for handle in handles_list:
  one_handle = get_tweets(handle)
  for each_tweet in one_handle:
    mega_list.append(each_tweet)

tweet_file = open("megafile.bin","wb")
pickle.dump(mega_list,tweet_file)
tweet_file.close()

In [ ]:
print(json.dumps(mega_list[0]._json))

{"created_at": "Fri Mar 06 12:40:34 +0000 2020", "id": 1235908129432322054, "id_str": "1235908129432322054", "text": "RT @DcpNorthDelhi: A desperate robber Karan@ kanni  has been arrested by team of Special staff along with 1 Country made weapon and 2 round\u2026", "truncated": false, "entities": {"hashtags": [], "symbols": [], "user_mentions": [{"screen_name": "DcpNorthDelhi", "name": "DCP North Delhi", "id": 2603573094, "id_str": "2603573094", "indices": [3, 17]}], "urls": []}, "source": "<a href=\"https://about.twitter.com/products/tweetdeck\" rel=\"nofollow\">TweetDeck</a>", "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_reply_to_user_id": null, "in_reply_to_user_id_str": null, "in_reply_to_screen_name": null, "user": {"id": 1850705408, "id_str": "1850705408", "name": "Delhi Police", "screen_name": "DelhiPolice", "location": "New Delhi, India", "description": "Official Twitter account of Delhi Police | Pls do not report crime here | In case of emergency #Dial

In [ ]:
# Print to CSV
o_file = open('tweetdump.csv', 'w')
f = csv.writer(o_file, delimiter =',')
f.writerow(["id", "date", "user", "text"])

for tweet in mega_list:
        tid = tweet.id
        date = tweet.created_at
        name = tweet.user.screen_name.strip()
        if tweet.retweeted:
          text = tweet.retweeted_status.text.strip()
        else:
          text = tweet.text.strip()
        f.writerow([tid, date, name, text])
o_file.close()

# Making the dataset

In [ ]:
ghazframe = pd.read_csv('tweetghaz.csv')


In [ ]:
dumpframe = pd.read_csv('tweetdump.csv')
dumpframe = dumpframe.loc[539:850]

In [ ]:
maindf = pd.concat([ghazframe,dumpframe],axis=0)
maindf

,id,date,user,text
0,1.235908e+18,2020-03-06 12:40:34,DelhiPolice,RT @DcpNorthDelhi: A desperate robber Karan@ k...
1,1.235908e+18,2020-03-06 12:39:55,DelhiPolice,"RT @DCPSouthDelhi: Various lectures,interactiv..."
2,1.235900e+18,2020-03-06 12:06:20,DelhiPolice,The team has been rewarded by Spl. CP/Operatio...
3,1.235900e+18,2020-03-06 12:06:19,DelhiPolice,"But displaying restraint as well as courage, p..."
4,1.235900e+18,2020-03-06 12:06:19,DelhiPolice,"Finding being cornered, the robbers abandoned ..."
...,...,...,...,...
846,1.238871e+18,2020-03-14 16:54:59,ranjitdash,@HMOIndia @DelhiPolice @ZeeNews\n@sudhirchaudh...
847,1.238871e+18,2020-03-14 16:53:02,samchaba9,@FreeYoursIndia @KhanAmanatullah @indiamusings...
848,1.238869e+18,2020-03-14 16:45:11,HariomS14730778,RT @milindsmarathe: Now the DCP Crime branch o...
849,1.238857e+18,2020-03-14 15:57:12,NeerjaAnand,@minupuri_r @BDUTT @DelhiPolice I am only obje...


## Feature Extraction

In [ ]:
# Hashtags
def hashtags(text):
  r = re.compile(r'\B(\#[a-zA-Z]+\b)(?!;)')
  hash_list = r.findall(text)
  if not hash_list:
    return "NaN" 
  # hashes = ''
  # for h in hash_list:
  #   hashes+=(h + " ,")
  return hash_list

# hashtags_dict = {}
# for tweet in mega_list:
#         hashtags = tweet.entities.get('hashtags')
#         for hashtag in hashtags:
#             if hashtag['text'] in hashtags_dict.keys():
#                 hashtags_dict[hashtag['text']] += 1
#             else:
#                 hashtags_dict[hashtag['text']] = 1

# answers = sorted(hashtags_dict, key=hashtags_dict.get, reverse=True)
# for i in range(0,len(answers)):
#   print( str(i+1) + ") " + answers[i] + ": "+str(hashtags_dict[answers[i]]))

maindf['hashtags'] = maindf['text'].apply(hashtags)
maindf

,id,date,user,text,hashtags,mentions
0,1.235908e+18,2020-03-06 12:40:34,DelhiPolice,RT @DcpNorthDelhi: A desperate robber Karan@ k...,NaN,[@DcpNorthDelhi]
1,1.235908e+18,2020-03-06 12:39:55,DelhiPolice,"RT @DCPSouthDelhi: Various lectures,interactiv...",NaN,[@DCPSouthDelhi]
2,1.235900e+18,2020-03-06 12:06:20,DelhiPolice,The team has been rewarded by Spl. CP/Operatio...,NaN,[]
3,1.235900e+18,2020-03-06 12:06:19,DelhiPolice,"But displaying restraint as well as courage, p...",NaN,[]
4,1.235900e+18,2020-03-06 12:06:19,DelhiPolice,"Finding being cornered, the robbers abandoned ...",NaN,[]
...,...,...,...,...,...,...
846,1.238871e+18,2020-03-14 16:54:59,ranjitdash,@HMOIndia @DelhiPolice @ZeeNews\n@sudhirchaudh...,NaN,"[@HMOIndia, @DelhiPolice, @ZeeNews, @sudhircha..."
847,1.238871e+18,2020-03-14 16:53:02,samchaba9,@FreeYoursIndia @KhanAmanatullah @indiamusings...,NaN,"[@FreeYoursIndia, @KhanAmanatullah, @indiamusi..."
848,1.238869e+18,2020-03-14 16:45:11,HariomS14730778,RT @milindsmarathe: Now the DCP Crime branch o...,NaN,"[@milindsmarathe, @DelhiPolice, @aishe_ghosh]"
849,1.238857e+18,2020-03-14 15:57:12,NeerjaAnand,@minupuri_r @BDUTT @DelhiPolice I am only obje...,NaN,"[@minupuri_r, @BDUTT, @DelhiPolice]"


In [ ]:
# Mentions
def mentions(text):
  r = re.compile(r'@[A-Za-z0-9_-]*[A-Za-z0-9_-]')
  mention_list = r.findall(text)
  # hashes = ''
  # for h in mention_list:
  #   hashes+=(h + " ,")
  if not mention_list:
    return "NaN"
  return mention_list
  # mentions_dict = {}
  # for tweet in mega_list:
  #         mentions = tweet.entities.get('user_mentions')
  #         if mentions:
  #           for mention in mentions:         
  #               if mention['screen_name'] in mentions_dict.keys():
  #                   mentions_dict[mention['screen_name']] += 1
  #               else:
  #                   mentions_dict[mention['screen_name']] = 1

# answers = sorted(mentions_dict, key=mentions_dict.get, reverse=True)
# for i in range(0,len(answers)):
#   print( str(i+1) + ") @" + answers[i] + ": "+str(mentions_dict[answers[i]]))

maindf['mentions'] = maindf['text'].apply(mentions)
maindf

,id,date,user,text,hashtags,mentions
0,1.235908e+18,2020-03-06 12:40:34,DelhiPolice,RT @DcpNorthDelhi: A desperate robber Karan@ k...,NaN,[@DcpNorthDelhi]
1,1.235908e+18,2020-03-06 12:39:55,DelhiPolice,"RT @DCPSouthDelhi: Various lectures,interactiv...",NaN,[@DCPSouthDelhi]
2,1.235900e+18,2020-03-06 12:06:20,DelhiPolice,The team has been rewarded by Spl. CP/Operatio...,NaN,NaN
3,1.235900e+18,2020-03-06 12:06:19,DelhiPolice,"But displaying restraint as well as courage, p...",NaN,NaN
4,1.235900e+18,2020-03-06 12:06:19,DelhiPolice,"Finding being cornered, the robbers abandoned ...",NaN,NaN
...,...,...,...,...,...,...
846,1.238871e+18,2020-03-14 16:54:59,ranjitdash,@HMOIndia @DelhiPolice @ZeeNews\n@sudhirchaudh...,NaN,"[@HMOIndia, @DelhiPolice, @ZeeNews, @sudhircha..."
847,1.238871e+18,2020-03-14 16:53:02,samchaba9,@FreeYoursIndia @KhanAmanatullah @indiamusings...,NaN,"[@FreeYoursIndia, @KhanAmanatullah, @indiamusi..."
848,1.238869e+18,2020-03-14 16:45:11,HariomS14730778,RT @milindsmarathe: Now the DCP Crime branch o...,NaN,"[@milindsmarathe, @DelhiPolice, @aishe_ghosh]"
849,1.238857e+18,2020-03-14 15:57:12,NeerjaAnand,@minupuri_r @BDUTT @DelhiPolice I am only obje...,NaN,"[@minupuri_r, @BDUTT, @DelhiPolice]"


In [ ]:
# Phone Numbers
def phones(text):
  phones_list = []
  r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{7})')
  # texts = ''
  # for tweet in mega_list:
  #   texts += tweet.text
  phones_list = r.findall(text)
  if not phones_list:
    return "NaN"
  # print(texts)
  return phones_list

# No phone numbers in collected tweets
maindf['phones'] = maindf['text'].apply(phones)
maindf

,id,date,user,text,hashtags,mentions,phones
0,1.235908e+18,2020-03-06 12:40:34,DelhiPolice,RT @DcpNorthDelhi: A desperate robber Karan@ k...,NaN,[@DcpNorthDelhi],NaN
1,1.235908e+18,2020-03-06 12:39:55,DelhiPolice,"RT @DCPSouthDelhi: Various lectures,interactiv...",NaN,[@DCPSouthDelhi],NaN
2,1.235900e+18,2020-03-06 12:06:20,DelhiPolice,The team has been rewarded by Spl. CP/Operatio...,NaN,NaN,NaN
3,1.235900e+18,2020-03-06 12:06:19,DelhiPolice,"But displaying restraint as well as courage, p...",NaN,NaN,NaN
4,1.235900e+18,2020-03-06 12:06:19,DelhiPolice,"Finding being cornered, the robbers abandoned ...",NaN,NaN,NaN
...,...,...,...,...,...,...,...
846,1.238871e+18,2020-03-14 16:54:59,ranjitdash,@HMOIndia @DelhiPolice @ZeeNews\n@sudhirchaudh...,NaN,"[@HMOIndia, @DelhiPolice, @ZeeNews, @sudhircha...",NaN
847,1.238871e+18,2020-03-14 16:53:02,samchaba9,@FreeYoursIndia @KhanAmanatullah @indiamusings...,NaN,"[@FreeYoursIndia, @KhanAmanatullah, @indiamusi...",NaN
848,1.238869e+18,2020-03-14 16:45:11,HariomS14730778,RT @milindsmarathe: Now the DCP Crime branch o...,NaN,"[@milindsmarathe, @DelhiPolice, @aishe_ghosh]",NaN
849,1.238857e+18,2020-03-14 15:57:12,NeerjaAnand,@minupuri_r @BDUTT @DelhiPolice I am only obje...,NaN,"[@minupuri_r, @BDUTT, @DelhiPolice]",NaN


In [ ]:
# Vehicles
def vehicles(text):
  vehicles_list = []
  r = re.compile(r'^[A-Z]{2}[ -][0-9]{1,2}(?: [A-Z])?(?: [A-Z]*)? [0-9]{4}$')
  # texts = ''
  # for tweet in mega_list:
  #   texts += tweet.text
  vehicles_list = r.findall(text)
  if not vehicles_list:
    return "NaN"
  # print(texts)
  return vehicles_list

maindf['vehicles'] = maindf['text'].apply(vehicles)
maindf

,id,date,user,text,hashtags,mentions,phones,vehicles
0,1.235908e+18,2020-03-06 12:40:34,DelhiPolice,RT @DcpNorthDelhi: A desperate robber Karan@ k...,NaN,[@DcpNorthDelhi],NaN,NaN
1,1.235908e+18,2020-03-06 12:39:55,DelhiPolice,"RT @DCPSouthDelhi: Various lectures,interactiv...",NaN,[@DCPSouthDelhi],NaN,NaN
2,1.235900e+18,2020-03-06 12:06:20,DelhiPolice,The team has been rewarded by Spl. CP/Operatio...,NaN,NaN,NaN,NaN
3,1.235900e+18,2020-03-06 12:06:19,DelhiPolice,"But displaying restraint as well as courage, p...",NaN,NaN,NaN,NaN
4,1.235900e+18,2020-03-06 12:06:19,DelhiPolice,"Finding being cornered, the robbers abandoned ...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
846,1.238871e+18,2020-03-14 16:54:59,ranjitdash,@HMOIndia @DelhiPolice @ZeeNews\n@sudhirchaudh...,NaN,"[@HMOIndia, @DelhiPolice, @ZeeNews, @sudhircha...",NaN,NaN
847,1.238871e+18,2020-03-14 16:53:02,samchaba9,@FreeYoursIndia @KhanAmanatullah @indiamusings...,NaN,"[@FreeYoursIndia, @KhanAmanatullah, @indiamusi...",NaN,NaN
848,1.238869e+18,2020-03-14 16:45:11,HariomS14730778,RT @milindsmarathe: Now the DCP Crime branch o...,NaN,"[@milindsmarathe, @DelhiPolice, @aishe_ghosh]",NaN,NaN
849,1.238857e+18,2020-03-14 15:57:12,NeerjaAnand,@minupuri_r @BDUTT @DelhiPolice I am only obje...,NaN,"[@minupuri_r, @BDUTT, @DelhiPolice]",NaN,NaN


In [ ]:
maindf.to_csv('dataset.csv')

In [ ]:
# Ghaziabad

ghaz_list = []
r = re.compile(r'Ghaziabad|ghaziabad|gzb|Ghzb|Gaziabad')
# r = re.compile(r'|\w*abad\b)
texts = ''
for tweet in mega_list:
  texts += tweet.text
ghaz_list = r.findall(texts)
# print(texts)
print(ghaz_list)

['Ghaziabad']


In [ ]:
# Getting rumors

rumor_tweets = []
for tweet in tweepy.Cursor(api.search,q="rumour #DelhiRiots", since="2020-02-01").items():
  rumor_tweets.append(tweet)
for tweet in tweepy.Cursor(api.search,q="rumour #DelhiViolence", since="2020-02-01").items():
  rumor_tweets.append(tweet)
for tweet in tweepy.Cursor(api.search,q="rumour #DelhiViolence2020", since="2020-02-01").items():
  rumor_tweets.append(tweet)
for tweet in tweepy.Cursor(api.search,q="rumour @DelhiPolice", since="2020-02-01").items():
  rumor_tweets.append(tweet)
for tweet in tweepy.Cursor(api.search,q="rumour @DelhiPolice", since="2020-02-01").items():
  rumor_tweets.append(tweet)

In [ ]:
o_file = open('tweetghaz.csv', 'a')
f = csv.writer(o_file, delimiter =',')

violent_tweets = []
for tweet in tweepy.Cursor(api.search,q="ghaziabad @DelhiPolice", since="2020-02-01").items():
  violent_tweets.append(tweet)
  tid = tweet.id
  date = tweet.created_at
  name = tweet.user.screen_name.strip()
  text = tweet.text.strip()
  f.writerow([tid, date, name, text])
  print(tweet.text)

o_file.close()

@hydcitypolice 👍👏 well done would request @Uppolice @ghaziabadpolice @DelhiPolice @dtptraffic @uptrafficpolice… https://t.co/jPiTaP8MlT
RT @DCPNWestDelhi: ASI Babulal &amp; Ct Paramjeet of PS Maurya Enclave have arrested accused Sahid r/o Ghaziabad and recovered entire jewellery…
RT @DCPNewDelhi: PS Chanakyapuri has solved two cases of theft by arresting two accused persons namely Avinash Kumar r/o Kannauj and Imran…
RT @DCPNWestDelhi: ASI Babulal &amp; Ct Paramjeet of PS Maurya Enclave have arrested accused Sahid r/o Ghaziabad and recovered entire jewellery…
RT @MyNationPrasadv: @iamarunjain @ipsnaithani @SspGhaziabad @igrangemeerut @adgzonemeerut @Uppolice @ghaziabadpolice @dm_ghaziabad @dgpup…
RT @DCPNewDelhi: PS Chanakyapuri has solved two cases of theft by arresting two accused persons namely Avinash Kumar r/o Kannauj and Imran…
RT @MyNationPrasadv: @iamarunjain @ipsnaithani @SspGhaziabad @igrangemeerut @adgzonemeerut @Uppolice @ghaziabadpolice @dm_ghaziabad @dgpup…
RT @MyNationPr

In [ ]:
for tweet in violent_tweets:
  text = tweet.full_text
  tid = tweet.id
  date = tweet.created_at
  name = tweet.user.screen_name.strip()
  print(tid,date,name,text)

AttributeError: ignored